In [5]:
import numpy as np
import pandas as pd
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
from openai import OpenAI


In [6]:
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")
embed = SentenceTransformer('bert-base-nli-mean-tokens')

# reading file

In [7]:
def read_doc(csv_file):
    loader = CSVLoader(file_path=csv_file,
        csv_args={
        'delimiter': ',',
        'quotechar': '"',
        'fieldnames': ['Link', 'content']
    })
    document = loader.load()
    return document

In [8]:
document = read_doc("clean_data.csv")
print(document[2].page_content)
# print(document)



Link: 1
content: ACHIEVEMENTS  Student Awards (Extra Curricular)  Student Awards (Extra Curricular) MIRAZ 24 (Rann-Neeti 24) The MIRAZ24 (Rann-Neeti24), a sports event, was hosted at the Indian Institute of Technology (IIT), Mandi, from April 12th to April 14th, 2024. The event aimed to unite talented athletes from various institutes across the nation to compete in a variety of sports disciplines. Our colleges Volleyball (Boys & Girls), Table Tennis (Boys & Girls), Athletics (Boys & Girls), Football (Boys), Cricket (Boys), and Basketball (Girls) teams participated in this tournament. Remarkably, the AKGEC sports team clinched an impressive tally of 17 Gold, 7 Silver, and 8 Bronze medals, securing the overall champions trophy at the second position. Additionally, they achieved the overall Athletics trophy at the first position, showcasing their exceptional sporting prowess and dedication. ATHLEMA Sports Tournament 2023 (Kho-Kho: Boys) Netaji Subhash Sports Complex, Sarita Vihar has been

# chanking doc

In [9]:
# funtion for chunk
def chunk_data(docs, size=1000, overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=overlap)
    doc = text_splitter.split_documents(docs)
    return doc

In [10]:
docs = chunk_data(document)
print(type(docs[0].page_content))
print(len(docs))
df = pd.DataFrame(docs[3])
df.to_csv("chunk_part.csv")
# print(docs[3].page_content)

<class 'str'>
1157


# creating embed

In [11]:
# result = client.embeddings.create(input = "hello world", model="text-embedding-mxbai-embed-large-v1")
# print(type(result))
# print(len(result.data[0].embedding))
# print(result)

In [12]:
# def get_embedding(text, model="text-embedding-mxbai-embed-large-v1"):
#     text = text.replace("\n", " ")
#     return client.embeddings.create(input = [text], model=model).data[0].embedding
# embeddings_list = []
# for i in range(len(docs)):
#     resposnse = get_embedding(docs[i].page_content)
#     embeddings_list.append(resposnse)
# embeddings_arr = np.array(embeddings_list).astype('float32')


In [13]:
# dimension = len(embeddings_list[0])
# index = faiss.IndexFlatL2(dimension)
# index.add(embeddings_arr)

In [14]:
# data = {
#     'Sentence': [
#         'AI is transforming the world.',
#         'Pandas is a powerful data analysis library.',
#         'Python is a versatile programming language.',
#         'Machine learning is a subset of AI.',
#         'Data science combines domain expertise, programming skills, and knowledge of math and statistics.',
#         'Cloud computing offers scalable resources over the internet.',
#         'Blockchain technology ensures secure and transparent transactions.',
#         'Deep learning models require vast amounts of data.',
#         'Cybersecurity is crucial for protecting digital assets.',
#         'Natural language processing enables computers to understand human language.',
#         'Art can be a powerful form of expression.',
#         'Renewable energy sources are essential for sustainability.',
#         'Sports bring people together and promote physical health.',
#         'Cooking can be a creative and enjoyable activity.',
#         'Music has the power to evoke emotions and memories.'
#     ],
#     'Category': [
#         'Technology',
#         'Library',
#         'Programming',
#         'Technology',
#         'Data Science',
#         'Technology',
#         'Finance',
#         'Data Science',
#         'Security',
#         'Technology',
#         'Art',
#         'Environment',
#         'Sports',
#         'Lifestyle',
#         'Music'
#     ]
# }
# dff = pd.DataFrame(data)
# len(dff["Sentence"])

In [15]:
# df = pd.DataFrame(data)
# print(df.shape)
enbedding_list = []
for i in range(len(docs)):
    resposnse = embed.encode(docs[i].page_content)
    enbedding_list.append(resposnse)
embeddings_df_arr = np.array(enbedding_list).astype('float32')
print(embeddings_df_arr.shape)
# print(embeddings_df_arr[0])


(1157, 768)


In [16]:
vector_dimension = embeddings_df_arr.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
print(len(embeddings_df_arr))
print(index.ntotal)
faiss.normalize_L2(embeddings_df_arr)
index.add(embeddings_df_arr)

1157
0


In [62]:
text = 'where is this college is locatted?'
text_embed = embed.encode([text])
text_arr = np.array(text_embed)
faiss.normalize_L2(text_arr)

# print(text_arr)
print(text_arr.shape)


(1, 768)


In [63]:
distance, idx = index.search(text_arr, k=15)
# rdf = pd.DataFrame({'distance':distance[0], 'idx':idx[0]})
# mdf = pd.merge(rdf, df,left_on='idx', right_index=True)
# mdf
vel = len(idx[0])
print(vel)
# print(docs[vel])
print(distance)
string=[]
for i in range(len(idx[0])):
    # print(docs[idx[0][i]])
    string.append((docs[idx[0][i]].page_content))
# print(docs[idx[0][0]])
# contents=docs[idx[0][1]].page_content
# print(type(docs[idx[0][1]].page_content))
context=" ".join(string)
# print(context)
file_path = "string.txt"

# Open the file in write mode and save the string
with open(file_path, 'w') as file:
    file.write(context)



15
[[0.68879104 0.8939587  0.8953505  0.90834683 0.9207142  0.9458928
  0.95949936 0.969906   0.98160124 0.9835181  0.9875246  0.9915896
  0.99509335 0.9956958  0.99670607]]


# chat call

In [64]:
#funtion for calling chat model
def call_openai_chat_model(api_key, model, user_input, context):
    OpenAI.api_key = api_key
    
    # Create the conversation messages
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a college chatbot who have to provide information about the college provided as context"},
            {"role": "assistant", "content": context},
            {"role": "user", "content": user_input}
        ],
        max_tokens=200,
        stream=True
    )

    # return completion.choices[0].message
    for chunk in completion:
        print(chunk.choices[0].delta.content)

In [65]:
print(context)

college. ISTE COORDINATORS HEAD COORDINATOR ADITYA PATEL ME CORE COORDINATOR MUKUL GOYAL IT CORE COORDINATOR PULKIT SAXENA CE STUDENT SECRETARY AVINASH RAI CSE CORE COORDINATOR VIVEK SINGH CSE CORE COORDINATOR UTKARSH VARMA IT CORE COORDINATOR SWETANK ASIT SRIVASTAVA ECE CORE COORDINATOR ARIHANT JAIN IT 2. Online transfer through NEFT/RTGS. Bank Account details of College for online payment are as under: Name of the Beneficiary : Ajay Kumar Garg Engineering College Name of the Bank : Kotak Mahindra Bank Ltd., Navyug Market, Ghaziabad -201001 (U.P.)-INDIA Bank Account No. : 508010250461 (Saving Bank Account) RTGS/NEFT/IFSC Code : KKBK0005295 Note : This Fee Structure is a demand letter from the college and valid for Bank Loan. TUITION FEE FOR THE M.TECH STUDENTS TO BE ADMITTED DURING THE SESSION 2024-25 M.TECH 1st Year (2024-25) Particulars I SEM (Amount in Rs) II SEM (Amount in Rs) TOTAL (Amount in Rs) Registration Fees 1500 1500 Tuition & Development Fees 55628 55628 111256 Exam & Enr

In [66]:

# with open('context.txt', 'r') as file:
#     # Read the content of the file and store it in a string
#     file_content = file.read()

In [67]:
call_openai_chat_model(api_key="no-needed", model="deepseek-r1-distill-llama-8b", user_input=text, context=context)


<think>


Okay
,
 so
 the
 user
 is
 asking
 where
 the
 Aj
ay
 Kumar
 G
arg
 Engineering
 College
 is
 located
.
 Let
 me
 look
 through
 the
 context
 provided
.


First
,
 I
 see
 mentions
 of
 Gh
azi
abad
 and
 UP
 (
U
tt
ar
 Pradesh
)
 in
 the
 bank
 account
 details
 and
 other
 sections
.
 That
 gives
 me
 a
 clue
 about
 the
 location
.
 


In
 the
 merit
 awards
 section
,
 I
 notice
 some
 roll
 numbers
 start
 with
 '
150
27
'
 which
 probably
 corresponds
 to
 Gh
azi
abad
's
 area
 code
.
 Also
,
 there
 are
 references
 to
 "
G
hazi
abad
 -
201
001
 (
U
.P
.)
-
IND
IA
"
 in
 the
 bank
 details
,
 confirming
 the
 location
.


So
 putting
 it
 all
 together
,
 the
 college
 is
 likely
 located
 in
 Gh
azi
abad
,
 Uttar
 Pradesh
,
 India
.

</think>



Aj
ay
 Kumar
 G
arg
 Engineering
 College
 is
 located
 in
 **
G
hazi
abad
**,
 Uttar
 Pradesh
,
 India
.
 The
 bank
 account
 details
 and
 other
 references
 in
 the
 context
 also
 specify
 the
 location
 as
 Gh
azi
abad
 -